# Ensemble Learning

## Initial Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


### Encoding Data using `LabelEncoder`

In [14]:
# Create the LabelEncoder instance
le = LabelEncoder()

# Integer Encoding to Columns
# home_ownership,
le.fit(df["home_ownership"])
df["home_ownership"] = le.transform(df["home_ownership"])

# verification_status, 
le.fit(df["verification_status"])
df["verification_status"] = le.transform(df["verification_status"])

# pymnt_plan, 
le.fit(df["pymnt_plan"])
df["pymnt_plan"] = le.transform(df["pymnt_plan"])

# initial_list_status,
le.fit(df["initial_list_status"])
df["initial_list_status"] = le.transform(df["initial_list_status"])

# issue_d,
le.fit(df["issue_d"])
df["issue_d"] = le.transform(df["issue_d"])

# next_pymnt_d
le.fit(df["next_pymnt_d"])
df["next_pymnt_d"] = le.transform(df["next_pymnt_d"])

# application_type, 
le.fit(df["application_type"])
df["application_type"] = le.transform(df["application_type"])

# hardship_flag, 
le.fit(df["hardship_flag"])
df["hardship_flag"] = le.transform(df["hardship_flag"])

# debt_settlement_flag
le.fit(df["debt_settlement_flag"])
df["debt_settlement_flag"] = le.transform(df["debt_settlement_flag"])


## Split the Data into Training and Testing

In [15]:
# Create our features
X = df.copy()
X.drop('loan_status', axis=1, inplace=True)


# Create our target
y = df['loan_status'].to_frame()

In [16]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.805542,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.714932,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,1.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [17]:
# Check the balance of our target values
y.value_counts()

loan_status
low_risk       68470
high_risk        347
dtype: int64

In [18]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [19]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [20]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [21]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [36]:
# Resample the training data with the BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=500, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=500, random_state=1)

In [37]:
# Calculated the balanced accuracy score
y_pred_brf = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred_brf)

0.7465939297390918

In [38]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_brf)

array([[   62,    39],
       [ 2064, 15040]], dtype=int64)

In [39]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_brf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.61      0.88      0.06      0.73      0.53       101
   low_risk       1.00      0.88      0.61      0.93      0.73      0.55     17104

avg / total       0.99      0.88      0.62      0.93      0.73      0.55     17205



In [40]:
# List the features sorted in descending order by feature importance
sorted_list = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
sorted_list

[(0.07409111765679259, 'total_rec_prncp'),
 (0.060452179429257874, 'last_pymnt_amnt'),
 (0.0587755448341319, 'total_pymnt_inv'),
 (0.056425078071337946, 'total_pymnt'),
 (0.05568863621603222, 'total_rec_int'),
 (0.0314642513641179, 'int_rate'),
 (0.022287260867422106, 'issue_d'),
 (0.018252700530333846, 'mths_since_recent_inq'),
 (0.018095657822525677, 'out_prncp'),
 (0.01778795388126791, 'installment'),
 (0.01767555266074872, 'out_prncp_inv'),
 (0.017391806994796134, 'dti'),
 (0.01647339388054553, 'max_bal_bc'),
 (0.01603948717022679, 'tot_hi_cred_lim'),
 (0.01576232321037389, 'total_bal_ex_mort'),
 (0.015478900399805587, 'total_bc_limit'),
 (0.015286960425796066, 'mo_sin_old_rev_tl_op'),
 (0.014839642757212274, 'annual_inc'),
 (0.014793720776113064, 'total_bal_il'),
 (0.014685145707129213, 'bc_util'),
 (0.01426238659336151, 'avg_cur_bal'),
 (0.014179146414509123, 'revol_bal'),
 (0.014158367422512516, 'mo_sin_old_il_acct'),
 (0.01401495053523028, 'all_util'),
 (0.013996809185474414, '

### Easy Ensemble Classifier

In [41]:
# Train the Classifier
easy = EasyEnsembleClassifier(random_state=1)
easy.fit(X_train, y_train)

EasyEnsembleClassifier(random_state=1)

In [42]:
# Calculated the balanced accuracy score
y_pred_easy = easy.predict(X_test)
balanced_accuracy_score(y_test, y_pred_easy)

0.914139417332753

In [43]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_easy)

array([[   92,     9],
       [ 1413, 15691]], dtype=int64)

In [44]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_easy))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.06      0.91      0.92      0.11      0.91      0.84       101
   low_risk       1.00      0.92      0.91      0.96      0.91      0.84     17104

avg / total       0.99      0.92      0.91      0.95      0.91      0.84     17205



### Final Questions

1. Which model had the best balanced accuracy score?

   The Easy Ensemble Classifier had the best accuracy score.

2. Which model had the best recall score?

   The Easy Ensemble Classifier had the best recall score.

3. Which model had the best geometric mean score?

   The Easy Ensemble Classifier had the best geometric mean score.

4. What are the top three features?

   (0.07409111765679259, 'total_rec_prncp'),
   
   (0.060452179429257874, 'last_pymnt_amnt'),
   
   (0.0587755448341319, 'total_pymnt_inv')